# Cell 1: Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import triang, uniform, bernoulli
import matplotlib.pyplot as plt
import seaborn as sns
import numpy_financial as npf
from tqdm.auto import tqdm
from multiprocessing import Pool
from monte_carlo_functions import (
    run_monte_carlo_simulation,
    N_ITERATIONS, CURRENT_YEAR, COMMERCIAL_LIFE_YEARS, TREATMENT_AGE_LIMIT,
    PARAMS, WAVES, BASE_LAUNCH_YEAR_WAVE1
)

from multiprocessing import cpu_count
n_processes = cpu_count() - 1 

# Cell 2: Patient Population Data


In [2]:
# Load actual timeline_df from the provided CSV file
csv_file_path = "/Users/smeden/Desktop/Lowe Syndrome Collaborative/Walther Therapeutics/git/Models/populationsdata/Simulated_ls_pop_1950_2060.csv"
try:
    timeline_df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded patient data from: {csv_file_path}")

    # Ensure 'alive' column is boolean
    if 'alive' in timeline_df.columns:
        # Handle potential string representations of boolean before converting
        if timeline_df['alive'].dtype == 'object':
            timeline_df['alive'] = timeline_df['alive'].str.lower().map({'true': True, 'false': False, True: True, False: False}).fillna(False)
        timeline_df['alive'] = timeline_df['alive'].astype(bool)
        print("Converted 'alive' column to boolean.")
    else:
        print("Warning: 'alive' column not found. Eligibility checks might be affected.")

    print("\nPatient data (timeline_df) ready for use.")
    print("DataFrame Info:")
    timeline_df.info()
    print("\nDataFrame Head:")
    print(timeline_df.head())
    print(f"\nYears covered: {timeline_df['year'].min()} to {timeline_df['year'].max()}")
    if 'iso3' in timeline_df.columns:
        print(f"Countries in data (first 5 unique): {timeline_df['iso3'].unique()[:5]}...")
    else:
        print("Column 'iso3' is missing after attempts to rename.")


except FileNotFoundError:
    print(f"Error: The file was not found at {csv_file_path}")
    print("Please ensure the file path is correct.")
    print("Using an empty DataFrame as a fallback. The simulation will likely produce 0 for patient numbers.")
    timeline_df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred while loading or processing the CSV: {e}")
    print("Using an empty DataFrame as a fallback.")
    timeline_df = pd.DataFrame()


Successfully loaded patient data from: /Users/smeden/Desktop/Lowe Syndrome Collaborative/Walther Therapeutics/git/Models/populationsdata/Simulated_ls_pop_1950_2060.csv
Converted 'alive' column to boolean.

Patient data (timeline_df) ready for use.
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2397628 entries, 0 to 2397627
Data columns (total 7 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   year          int64 
 1   country       object
 2   region        object
 3   patient_uuid  object
 4   age           int64 
 5   alive         bool  
 6   iso3          object
dtypes: bool(1), int64(2), object(4)
memory usage: 112.0+ MB

DataFrame Head:
   year country    region                          patient_uuid  age  alive  \
0  1956      CD    Africa  59398283-cda8-4971-98e9-d89d46f701d7    0   True   
1  1956      LU    Europe  a159aeea-7a00-4e16-b9f7-fe5e613760a4    0   True   
2  1956      VE  Americas  b0ef5ec6-22f3-49a9-a626-9ce2c0f50cab    0   True

# Cell 3: Run Simulation

In [3]:
print(f"Starting Monte Carlo simulation with {N_ITERATIONS} iterations...")
simulation_results_df = run_monte_carlo_simulation(
    params=PARAMS,
    waves_config=WAVES,
    base_launch_year_w1=BASE_LAUNCH_YEAR_WAVE1,
    timeline_df=timeline_df,
    n_iterations=500,
    current_year=CURRENT_YEAR,
    commercial_life_years=COMMERCIAL_LIFE_YEARS,
    treatment_age_limit=TREATMENT_AGE_LIMIT,
    n_processes=n_processes
)
print("Monte Carlo simulation finished.")
print(simulation_results_df.head())

Starting Monte Carlo simulation with 1000 iterations...
Starting Monte Carlo simulation with 500 iterations...


Running simulations:   0%|          | 0/500 [00:05<?, ?it/s]

Monte Carlo simulation finished.
   iteration  project_succeeded  total_rd_cost_millions  \
0          0              False                6.362813   
1          1               True               27.851089   
2          2               True               25.233544   
3          3               True               20.915083   
4          4               True               25.170786   

   total_rd_duration_years  prv_obtained  prv_value_millions  odd_obtained  \
0                 5.902946         False            0.000000             1   
1                10.476096          True          127.560331             0   
2                11.261698          True          112.467110             1   
3                 9.428594          True          108.958652             1   
4                10.258494         False            0.000000             1   

                            annual_rd_costs_stream_M  \
0  [-0.5928307331712354, -0.5928307331712354, -0....   
1  [-0.9637077150464646, -0.962

In [4]:
simulation_results_df.to_csv('output/simulation_results_minus2discount_500_df.csv', index=False)

# Cell 7: Analyze and Visualize Results


In [5]:
if not simulation_results_df.empty:
    # Basic Statistics
    mean_rnpv = simulation_results_df['npv_usd_millions'].mean()
    median_rnpv = simulation_results_df['npv_usd_millions'].median()
    std_rnpv = simulation_results_df['npv_usd_millions'].std()
    prob_positive_npv = (simulation_results_df['npv_usd_millions'] > 0).mean() * 100
    
    successful_outcomes_df = simulation_results_df[simulation_results_df['project_succeeded'] == True].copy() 
    prob_project_success = len(successful_outcomes_df) / N_ITERATIONS * 100
    
    if not successful_outcomes_df.empty:
        mean_npv_if_success = successful_outcomes_df['npv_usd_millions'].mean()
        print(f"\nMean NPV (if project succeeds): ${mean_npv_if_success:,.2f} million")
    else:
        print("\nNo successful project outcomes in the simulation.")

    print(f"\n--- Simulation Results ---")
    print(f"Mean rNPV (overall project value from t=0): ${mean_rnpv:,.2f} million")
    print(f"Median rNPV: ${median_rnpv:,.2f} million")
    print(f"Standard Deviation of NPV: ${std_rnpv:,.2f} million")
    print(f"Probability of Overall Project Success (Launch): {prob_project_success:.2f}%")
    print(f"Overall Probability of Positive NPV: {prob_positive_npv:.2f}%")

    # NPV Distribution Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(simulation_results_df['npv_usd_millions'], kde=True, bins=50, ax=ax)
    ax.set_title(f'Distribution of Overall Project NPV (t=0) ({N_ITERATIONS} Iterations)')
    ax.set_xlabel('NPV (USD Millions)')
    ax.set_ylabel('Frequency')
    ax.axvline(mean_rnpv, color='r', linestyle='--', label=f'Mean rNPV: ${mean_rnpv:,.2f}M')
    ax.axvline(0, color='k', linestyle='-')
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.7)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    fig.patch.set_alpha(0.0)
    plt.savefig('output/npv_distribution.png', bbox_inches='tight', transparent=True)
    plt.close()

    # --- New Visualizations ---
    if not successful_outcomes_df.empty:
        # 1. Distribution of Total R&D Costs (Successful Runs)
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.histplot(successful_outcomes_df['total_rd_cost_millions'], kde=True, bins=30, ax=ax)
        ax.set_title('Distribution of Total R&D Costs (Successful Projects)')
        ax.set_xlabel('Total R&D Cost (USD Millions)')
        ax.set_ylabel('Frequency')
        ax.axvline(successful_outcomes_df['total_rd_cost_millions'].mean(), color='r', linestyle='--', 
                   label=f'Mean: ${successful_outcomes_df["total_rd_cost_millions"].mean():,.2f}M')
        ax.legend()
        ax.grid(True, linestyle=':', alpha=0.7)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)
        fig.patch.set_alpha(0.0)
        plt.savefig('output/rd_costs_distribution.png', bbox_inches='tight', transparent=True)
        plt.close()

        # 2. Distribution of Total R&D Durations (Successful Runs)
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.histplot(successful_outcomes_df['total_rd_duration_years'], kde=True, bins=30, ax=ax)
        ax.set_title('Distribution of Total R&D Durations (Successful Projects)')
        ax.set_xlabel('Total R&D Duration (Years)')
        ax.set_ylabel('Frequency')
        ax.axvline(successful_outcomes_df['total_rd_duration_years'].mean(), color='r', linestyle='--',
                   label=f'Mean: {successful_outcomes_df["total_rd_duration_years"].mean():,.2f} yrs')
        ax.legend()
        ax.grid(True, linestyle=':', alpha=0.7)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)
        fig.patch.set_alpha(0.0)
        plt.savefig('output/rd_duration_distribution.png', bbox_inches='tight', transparent=True)
        plt.close()

        # 3. Annual Patients Treated (Post-Launch for Successful Runs)
        patients_treated_annual_list = [s for s in successful_outcomes_df['annual_patients_treated'].dropna() if s is not None and len(s) > 0]
        if patients_treated_annual_list:
            max_len_patients = max(len(s) for s in patients_treated_annual_list)
            patients_treated_df_plot = pd.DataFrame([list(s) + [np.nan]*(max_len_patients - len(s)) for s in patients_treated_annual_list])

            mean_patients_treated = patients_treated_df_plot.mean()
            p10_patients_treated = patients_treated_df_plot.quantile(0.10)
            p90_patients_treated = patients_treated_df_plot.quantile(0.90)
            
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(mean_patients_treated.index, mean_patients_treated, label='Mean Patients Treated per Year', marker='o')
            ax.fill_between(mean_patients_treated.index, p10_patients_treated, p90_patients_treated, alpha=0.2, label='P10-P90 Range')
            ax.set_title('Annual Patients Treated (Post-Launch, Successful Projects)')
            ax.set_xlabel('Years Post-Launch')
            ax.set_ylabel('Number of Patients Treated')
            ax.legend()
            ax.grid(True, linestyle=':', alpha=0.7)
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_visible(False)
            fig.patch.set_alpha(0.0)
            plt.savefig('output/annual_patients.png', bbox_inches='tight', transparent=True)
            plt.close()
        else:
            print("No data for annual patients treated plot (successful runs).")

        # 4. Annual Sales (Post-Launch for Successful Runs)
        sales_annual_list = [s for s in successful_outcomes_df['annual_commercial_revenues_M'].dropna() if s is not None and len(s) > 0]
        if sales_annual_list:
            max_len_sales = max(len(s) for s in sales_annual_list)
            sales_df_plot = pd.DataFrame([list(s) + [np.nan]*(max_len_sales - len(s)) for s in sales_annual_list])
            
            mean_sales = sales_df_plot.mean()
            p10_sales = sales_df_plot.quantile(0.10)
            p90_sales = sales_df_plot.quantile(0.90)

            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(mean_sales.index, mean_sales, label='Mean Annual Sales (USD Millions)', marker='o')
            ax.fill_between(mean_sales.index, p10_sales, p90_sales, alpha=0.2, label='P10-P90 Range')
            ax.set_title('Annual Sales (Post-Launch, Successful Projects)')
            ax.set_xlabel('Years Post-Launch')
            ax.set_ylabel('Sales (USD Millions)')
            ax.legend()
            ax.grid(True, linestyle=':', alpha=0.7)
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_visible(False)
            fig.patch.set_alpha(0.0)
            plt.savefig('output/annual_sales.png', bbox_inches='tight', transparent=True)
            plt.close()
        else:
            print("No data for annual sales plot (successful runs).")

        # 5. ODD/PRV Impact on NPV (Box Plots for Successful Runs)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        if 'odd_obtained' in successful_outcomes_df.columns:
            sns.boxplot(x='odd_obtained', y='npv_usd_millions', data=successful_outcomes_df, ax=ax1)
            ax1.set_title('NPV by ODD Obtainment (Successful Projects)')
            ax1.set_xlabel('ODD Obtained')
            ax1.set_ylabel('NPV (USD Millions)')
            ax1.spines['top'].set_visible(False)
            ax1.spines['left'].set_visible(False)
        
        if 'prv_obtained' in successful_outcomes_df.columns:
            sns.boxplot(x='prv_obtained', y='npv_usd_millions', data=successful_outcomes_df, ax=ax2)
            ax2.set_title('NPV by PRV Obtainment (Successful Projects)')
            ax2.set_xlabel('PRV Obtained & Sold')
            ax2.set_ylabel('NPV (USD Millions)')
            ax2.spines['top'].set_visible(False)
            ax2.spines['left'].set_visible(False)
        
        fig.patch.set_alpha(0.0)
        plt.tight_layout()
        plt.savefig('output/odd_prv_impact.png', bbox_inches='tight', transparent=True)
        plt.close()

    # 6. Project Lifecycle Net Cash Flow (All Simulations)
    all_cash_flows_list = [s for s in simulation_results_df['full_project_cash_flow_stream_M'].dropna() if s is not None and len(s) > 0]
    if all_cash_flows_list:
        max_len_full_cf = max(len(s) for s in all_cash_flows_list)
        padded_cash_flows = [list(s) + [0] * (max_len_full_cf - len(s)) for s in all_cash_flows_list]
        full_cash_flow_df = pd.DataFrame(padded_cash_flows)

        mean_full_cf = full_cash_flow_df.mean()
        p10_full_cf = full_cash_flow_df.quantile(0.10)
        p90_full_cf = full_cash_flow_df.quantile(0.90)
        
        fig, ax = plt.subplots(figsize=(14, 7))
        ax.plot(mean_full_cf.index, mean_full_cf, label='Mean Annual Net Cash Flow', marker='.')
        ax.fill_between(mean_full_cf.index, p10_full_cf, p90_full_cf, alpha=0.2, label='P10-P90 Range')
        ax.set_title('Project Lifecycle Annual Net Cash Flow (All Simulations)')
        ax.set_xlabel('Project Year (from t=0)')
        ax.set_ylabel('Net Cash Flow (USD Millions)')
        ax.axhline(0, color='k', linestyle='-', linewidth=0.8)
        ax.legend()
        ax.grid(True, linestyle=':', alpha=0.7)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)
        fig.patch.set_alpha(0.0)
        plt.savefig('output/lifecycle_cash_flow.png', bbox_inches='tight', transparent=True)
        plt.close()

        # 7. Cumulative Project Net Cash Flow
        cumulative_full_cf_df = full_cash_flow_df.cumsum(axis=1)
        mean_cumulative_cf = cumulative_full_cf_df.mean()
        p10_cumulative_cf = cumulative_full_cf_df.quantile(0.10)
        p90_cumulative_cf = cumulative_full_cf_df.quantile(0.90)

        fig, ax = plt.subplots(figsize=(14, 7))
        ax.plot(mean_cumulative_cf.index, mean_cumulative_cf, label='Mean Cumulative Net Cash Flow', marker='.')
        ax.fill_between(mean_cumulative_cf.index, p10_cumulative_cf, p90_cumulative_cf, alpha=0.2, label='P10-P90 Range')
        ax.set_title('Cumulative Project Net Cash Flow (All Simulations)')
        ax.set_xlabel('Project Year (from t=0)')
        ax.set_ylabel('Cumulative Net Cash Flow (USD Millions)')
        ax.axhline(0, color='k', linestyle='-', linewidth=0.8)
        ax.legend()
        ax.grid(True, linestyle=':', alpha=0.7)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)
        fig.patch.set_alpha(0.0)
        plt.savefig('output/cumulative_cash_flow.png', bbox_inches='tight', transparent=True)
        plt.close()
    else:
        print("No cash flow streams found for lifecycle plots.")
        
    # --- Plotting rNPV at Phase Gates (for successful runs) ---
    if not successful_outcomes_df.empty:
        phase_rnpv_cols = ['rnpv_at_preclinical_start', 'rnpv_at_phase1_2_start', 
                           'rnpv_at_phase3_start', 'rnpv_at_regulatory_start', 'rnpv_at_launch']
        
        # Check if columns exist before trying to plot
        plot_phase_rnpv = True
        for col in phase_rnpv_cols:
            if col not in successful_outcomes_df.columns:
                print(f"Warning: Column {col} not found in successful_outcomes_df. Skipping phase gate rNPV plots.")
                plot_phase_rnpv = False
                break
        
        if plot_phase_rnpv:
            fig, ax = plt.subplots(figsize=(15, 8))
            sns.boxplot(data=successful_outcomes_df[phase_rnpv_cols], ax=ax)
            ax.set_title('Distribution of rNPV at Start of Each Phase (Successful Projects)')
            ax.set_ylabel('rNPV (USD Millions)')
            ax.set_xlabel('Phase Start')
            plt.xticks(rotation=45, ha="right")
            ax.grid(True, linestyle=':', alpha=0.7)
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_visible(False)
            fig.patch.set_alpha(0.0)
            plt.tight_layout()
            plt.savefig('output/phase_gate_rnpv.png', bbox_inches='tight', transparent=True)
            plt.close()

            print("\nMean rNPV at the start of each phase (for projects that eventually launch):")
            for col in phase_rnpv_cols:
                mean_val = successful_outcomes_df[col].mean()
                print(f"  Mean {col.replace('_', ' ').title()}: ${mean_val:,.2f} million")
    
    # Other statistics from original cell
    if not successful_outcomes_df.empty:
        print(f"\nAverage R&D Duration (if successful): {successful_outcomes_df['total_rd_duration_years'].mean():.2f} years")
        print(f"Average R&D Cost (if successful): ${successful_outcomes_df['total_rd_cost_millions'].mean():,.2f} million")
        
        if 'odd_obtained' in successful_outcomes_df.columns:
            print(f"ODD Obtained in {successful_outcomes_df['odd_obtained'].mean()*100:.2f}% of successful runs")
        if 'prv_obtained' in successful_outcomes_df.columns:
            print(f"PRV Obtained in {successful_outcomes_df['prv_obtained'].mean()*100:.2f}% of successful runs")
            if not successful_outcomes_df[successful_outcomes_df['prv_obtained']==True].empty:
                 print(f"Average PRV Value (if obtained & sold): ${successful_outcomes_df[successful_outcomes_df['prv_obtained']==True]['prv_value_millions'].mean():,.2f} million")
    else:
        print("\nNo successful project outcomes to report R&D duration/cost or ODD/PRV stats for.")

else:
    print("Simulation results DataFrame is empty. Check for errors.")



Mean NPV (if project succeeds): $52.91 million

--- Simulation Results ---
Mean rNPV (overall project value from t=0): $28.84 million
Median rNPV: $40.27 million
Standard Deviation of NPV: $30.20 million
Probability of Overall Project Success (Launch): 29.20%
Overall Probability of Positive NPV: 58.40%

Mean rNPV at the start of each phase (for projects that eventually launch):
  Mean Rnpv At Preclinical Start: $39.46 million
  Mean Rnpv At Phase1 2 Start: $66.71 million
  Mean Rnpv At Phase3 Start: $153.56 million
  Mean Rnpv At Regulatory Start: $335.29 million
  Mean Rnpv At Launch: $446.40 million

Average R&D Duration (if successful): 9.97 years
Average R&D Cost (if successful): $22.49 million
ODD Obtained in 94.86% of successful runs
PRV Obtained in 48.29% of successful runs
Average PRV Value (if obtained & sold): $114.29 million


# Cell 9: Wave Statistics Analysis

In [6]:

# Phase Duration Timeline Analysis
if not simulation_results_df.empty:
    # Extract phase durations for successful outcomes
    successful_outcomes_df = simulation_results_df[simulation_results_df['project_succeeded'] == True].copy()
    
    if not successful_outcomes_df.empty:
        phase_durations = successful_outcomes_df[[
            'duration_preclinical_yrs',
            'duration_phase1_2_yrs',
            'duration_phase3_yrs',
            'duration_regulatory_yrs'
        ]]
        
        # Calculate statistics
        mean_durations = phase_durations.mean()
        ci_lower = phase_durations.quantile(0.025)  # 2.5th percentile for 95% CI
        ci_upper = phase_durations.quantile(0.975)  # 97.5th percentile for 95% CI
        
        # Create timeline plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Plot mean durations as points
        y_pos = np.arange(len(mean_durations))
        ax.scatter(mean_durations, y_pos, color='blue', s=100, zorder=3, label='Mean Duration')
        
        # Plot confidence intervals as horizontal lines
        for i, (lower, upper) in enumerate(zip(ci_lower, ci_upper)):
            ax.plot([lower, upper], [i, i], color='blue', alpha=0.3, linewidth=10)
        
        # Add cumulative timeline markers
        cumulative_means = np.cumsum(mean_durations)
        for i, cum_mean in enumerate(cumulative_means):
            ax.axvline(x=cum_mean, color='gray', linestyle='--', alpha=0.3)
            ax.text(cum_mean, -0.5, f'{cum_mean:.1f}y', ha='center', va='top')
        
        # Customize plot
        ax.set_yticks(y_pos)
        ax.set_yticklabels(['Preclinical', 'Phase 1/2', 'Phase 3', 'Regulatory'])
        ax.set_xlabel('Duration (Years)')
        ax.set_title('Phase Duration Timeline with 95% Confidence Intervals\n(Cumulative timeline shown with dashed lines)')
        ax.grid(True, linestyle=':', alpha=0.7)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        # Print statistics
        print("\nPhase Duration Statistics (Years):")
        print("--------------------------------")
        cumulative = 0
        for phase, mean, lower, upper in zip(phase_durations.columns, mean_durations, ci_lower, ci_upper):
            cumulative += mean
            print(f"\n{phase.replace('duration_', '').replace('_yrs', '').title()}:")
            print(f"  Mean: {mean:.1f} years")
            print(f"  95% CI: [{lower:.1f}, {upper:.1f}]")
            print(f"  Cumulative Timeline: {cumulative:.1f} years")
        
        plt.tight_layout()
        plt.savefig('output/phase_duration.png', bbox_inches='tight', transparent=True)
        plt.close()

# Wave Analysis
if not simulation_results_df.empty:
    successful_outcomes_df = simulation_results_df[simulation_results_df['project_succeeded'] == True].copy()
    
    if not successful_outcomes_df.empty:
        def analyze_wave_stats(successful_outcomes_df):
            wave_stats = {}
            for wave in [1, 2, 3]:
                sales_col = f'wave{wave}_annual_sales_M'
                patients_col = f'wave{wave}_annual_patients'
                year_col = f'wave{wave}_launch_year'
                
                if all(col in successful_outcomes_df.columns for col in [sales_col, patients_col, year_col]):
                    # Calculate peak sales and the year (offset) of peak for each run
                    def get_peak_and_year(sales_list):
                        if isinstance(sales_list, (list, np.ndarray)) and len(sales_list) > 0:
                            peak = max(sales_list)
                            peak_year = sales_list.index(peak)
                            return peak, peak_year
                        return 0, None

                    def get_half_peak_year(sales_list):
                        if isinstance(sales_list, (list, np.ndarray)) and len(sales_list) > 0:
                            peak = max(sales_list)
                            half_peak = 0.5 * peak
                            for i, val in enumerate(sales_list):
                                if val >= half_peak:
                                    return i
                        return None

                    peak_sales_and_years = successful_outcomes_df[sales_col].apply(get_peak_and_year)
                    wave_peak_sales = peak_sales_and_years.apply(lambda x: x[0])
                    wave_peak_years = peak_sales_and_years.apply(lambda x: x[1])

                    wave_peak_patients = successful_outcomes_df[patients_col].apply(lambda x: max(x) if isinstance(x, (list, np.ndarray)) and len(x) > 0 else 0)
                    
                    # Calculate stats
                    sales_stats = {
                        'mean': wave_peak_sales.mean(),
                        'ci_lower': np.percentile(wave_peak_sales, 2.5),
                        'ci_upper': np.percentile(wave_peak_sales, 97.5)
                    }
                    patients_stats = {
                        'mean': wave_peak_patients.mean(),
                        'ci_lower': np.percentile(wave_peak_patients, 2.5),
                        'ci_upper': np.percentile(wave_peak_patients, 97.5)
                    }
                    # Calculate average year of peak sales (relative to launch)
                    avg_peak_year = wave_peak_years.dropna().mean() if not wave_peak_years.dropna().empty else None

                    # Calculate average year to reach 50% of peak sales
                    half_peak_years = successful_outcomes_df[sales_col].apply(get_half_peak_year)
                    avg_half_peak_year = half_peak_years.dropna().mean() if not half_peak_years.dropna().empty else None

                    # Store in dictionary
                    wave_stats[f'Wave {wave}'] = {
                        'Peak Sales (USD M)': sales_stats,
                        'Peak Patients': patients_stats,
                        'Launch Year': successful_outcomes_df[year_col].mean(),
                        'Avg Peak Sales Year (relative to launch)': avg_peak_year,
                        'Avg 50% Peak Sales Year (relative to launch)': avg_half_peak_year
                    }
            return wave_stats
                
        wave_stats = analyze_wave_stats(successful_outcomes_df)
        print("\nWave Statistics Analysis:")
        print("------------------------")
        for wave, stats in wave_stats.items():
            print(f"\n{wave}:")
            print(f"Launch Year (Mean): Year {stats['Launch Year']:.1f}")
            sales_stats = stats['Peak Sales (USD M)']
            print(f"Peak Sales:")
            print(f"  Mean: ${sales_stats['mean']:.1f}M")
            print(f"  95% CI: [${sales_stats['ci_lower']:.1f}M, ${sales_stats['ci_upper']:.1f}M]")
            patient_stats = stats['Peak Patients']
            print(f"Peak Patients:")
            print(f"  Mean: {patient_stats['mean']:.0f}")
            print(f"  95% CI: [{patient_stats['ci_lower']:.0f}, {patient_stats['ci_upper']:.0f}]")
            if stats['Avg 50% Peak Sales Year (relative to launch)'] is not None:
                print(f"Average Year to Reach 50% of Peak Sales (relative to launch): {stats['Avg 50% Peak Sales Year (relative to launch)']:.1f}")
            else:
                print("Average Year to Reach 50% of Peak Sales (relative to launch): N/A")

# Calculate overall statistics across waves
if not simulation_results_df.empty and successful_outcomes_df is not None:
    # For each simulation, combine sales from all waves year by year
    def get_total_sales_by_year(row):
        # Initialize lists to store combined sales and patients for each year
        max_years = max(len(row.get(f'wave{i}_annual_sales_M', [])) for i in [1,2,3])
        total_sales_by_year = np.zeros(max_years)
        total_patients_by_year = np.zeros(max_years)
        
        # Add sales and patients from each wave
        for wave in [1,2,3]:
            wave_sales = row.get(f'wave{wave}_annual_sales_M', [])
            wave_patients = row.get(f'wave{wave}_annual_patients', [])
            
            # Add sales and patients year by year
            for year, (sales, patients) in enumerate(zip(wave_sales, wave_patients)):
                total_sales_by_year[year] += sales
                total_patients_by_year[year] += patients
        
        # Find the peak year and values
        peak_sales = np.max(total_sales_by_year)
        peak_year = np.argmax(total_sales_by_year)
        peak_patients = total_patients_by_year[peak_year]
        avg_price = peak_sales / peak_patients if peak_patients > 0 else 0
        
        # Calculate cumulative values
        cumulative_sales = np.sum(total_sales_by_year)
        cumulative_patients = np.sum(total_patients_by_year)
        
        return peak_sales, peak_year, peak_patients, avg_price, cumulative_sales, cumulative_patients

    # Calculate peak statistics across all simulations
    peak_stats = successful_outcomes_df.apply(get_total_sales_by_year, axis=1)
    peak_sales = [x[0] for x in peak_stats]
    peak_years = [x[1] for x in peak_stats]
    peak_patients = [x[2] for x in peak_stats]
    avg_prices = [x[3] for x in peak_stats]
    cumulative_sales = [x[4] for x in peak_stats]
    cumulative_patients = [x[5] for x in peak_stats]

    # Print results
    print(f"\nOverall Peak Statistics:")
    print(f"Peak Sales: ${np.mean(peak_sales):.1f}M (95% CI: ${np.percentile(peak_sales, 2.5):.1f}M - ${np.percentile(peak_sales, 97.5):.1f}M)")
    print(f"Peak Year: {2034 + np.mean(peak_years):.1f} (95% CI: {2034 + np.percentile(peak_years, 2.5):.1f} - {2034 + np.percentile(peak_years, 97.5):.1f})")
    print(f"Peak Patients: {np.mean(peak_patients):.0f} (95% CI: {np.percentile(peak_patients, 2.5):.0f} - {np.percentile(peak_patients, 97.5):.0f})")
    print(f"Average Price at Peak: ${np.mean(avg_prices):.1f}M (95% CI: ${np.percentile(avg_prices, 2.5):.1f}M - ${np.percentile(avg_prices, 97.5):.1f}M)")
    print(f"\nCumulative Values:")
    print(f"Total Cumulative Sales: ${np.mean(cumulative_sales):.1f}M (95% CI: ${np.percentile(cumulative_sales, 2.5):.1f}M - ${np.percentile(cumulative_sales, 97.5):.1f}M)")
    print(f"Total Cumulative Patients: {np.mean(cumulative_patients):.0f} (95% CI: {np.percentile(cumulative_patients, 2.5):.0f} - {np.percentile(cumulative_patients, 97.5):.0f})")


Phase Duration Statistics (Years):
--------------------------------

Preclinical:
  Mean: 1.9 years
  95% CI: [1.2, 2.7]
  Cumulative Timeline: 1.9 years

Phase1_2:
  Mean: 3.4 years
  95% CI: [2.4, 4.5]
  Cumulative Timeline: 5.3 years

Phase3:
  Mean: 3.6 years
  95% CI: [2.4, 4.6]
  Cumulative Timeline: 8.9 years

Regulatory:
  Mean: 1.1 years
  95% CI: [0.8, 1.4]
  Cumulative Timeline: 10.0 years

Wave Statistics Analysis:
------------------------

Wave 1:
Launch Year (Mean): Year 2034.5
Peak Sales:
  Mean: $188.8M
  95% CI: [$149.6M, $240.5M]
Peak Patients:
  Mean: 78
  95% CI: [66, 97]
Average Year to Reach 50% of Peak Sales (relative to launch): 1.0

Wave 2:
Launch Year (Mean): Year 2037.5
Peak Sales:
  Mean: $93.6M
  95% CI: [$73.4M, $122.7M]
Peak Patients:
  Mean: 39
  95% CI: [31, 48]
Average Year to Reach 50% of Peak Sales (relative to launch): 1.8

Wave 3:
Launch Year (Mean): Year 2039.5
Peak Sales:
  Mean: $25.0M
  95% CI: [$19.1M, $32.6M]
Peak Patients:
  Mean: 10
  95% 

In [11]:
df = pd.read_csv('output/simulation_results_base_df.csv')
df['project_succeeded']

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: project_succeeded, Length: 1000, dtype: bool

In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import ast

# ---------------------------------------------------------------------------
# 1. Load & prepare the data
# ---------------------------------------------------------------------------
df = pd.read_csv("output/simulation_results_plus10price1000_df.csv")
successful_projects = df.loc[df["project_succeeded"]]


def sum_revenues(revenue_list_str: str) -> float:
    try:
        return sum(ast.literal_eval(revenue_list_str))
    except Exception:  # noqa: BLE001
        return 0


successful_projects["cumulative_revenues"] = (
    successful_projects['annual_commercial_revenues_M'].apply(sum_revenues) / 1e3
)  # → billions

# ---------------------------------------------------------------------------
# 2. Build a custom two-colour gradient (light on the left, dark on the right)
# ---------------------------------------------------------------------------
grad_cmap = LinearSegmentedColormap.from_list(
    "cyan_blue_grad", ["#82E7F2", "#2E8CEE"]
)

# ---------------------------------------------------------------------------
# 3. Plot
# ---------------------------------------------------------------------------
plt.figure(figsize=(12, 6))
ax = sns.histplot(
    data=successful_projects["cumulative_revenues"],
    bins=30,
    kde=True,
    edgecolor="black",
    line_kws={"color": "#2E8CEE", "linewidth": 2},
)

# ── Re-colour the bars -------------------------------------------------------
bars = ax.patches
n_bars = len(bars)

# Sort right-to-left so bar 0 (index 0) is the right-most and gets the dark colour
for i, bar in enumerate(sorted(bars, key=lambda b: b.get_x(), reverse=False)):
    bar.set_facecolor(grad_cmap(i / n_bars))

# ── Minimalist styling -------------------------------------------------------
ax.set_title("Distribution of Cumulative Total Revenues (Successful Projects Only)")
ax.set_xlabel("Cumulative Revenue (Billion USD)")
ax.set_ylabel("Frequency")

# Remove grid, background and all spines
ax.grid(False)
ax.set_facecolor("none")
ax.figure.patch.set_facecolor("none")
for spine in ax.spines.values():
    spine.set_visible(False)

# (Optional) keep tick labels but remove tick marks themselves for a cleaner frame
ax.tick_params(left=False, bottom=False)

# ---------------------------------------------------------------------------
# 4. (Unchanged) summary statistics & annotations
# ---------------------------------------------------------------------------
mean_rev   = successful_projects["cumulative_revenues"].mean()
median_rev = successful_projects["cumulative_revenues"].median()
std_rev    = successful_projects["cumulative_revenues"].std()
p10        = successful_projects["cumulative_revenues"].quantile(0.10)
p90        = successful_projects["cumulative_revenues"].quantile(0.90)

ax.axvline(mean_rev,   color="#2E8CEE", linestyle="--", label=f"Mean:   ${mean_rev:.1f} B")
ax.axvline(median_rev, color="#82E7F2", linestyle="--", label=f"Median: ${median_rev:.1f} B")

stats_text = (
    "Summary Statistics (USD Billions):\n"
    f"Mean:   ${mean_rev:.1f}\n"
    f"Median: ${median_rev:.1f}\n"
    f"Std Dev:${std_rev:.1f}\n"
    f"10th P.:${p10:.1f}\n"
    f"90th P.:${p90:.1f}"
)
#ax.text(
#    0.95, 0.95, stats_text, transform=ax.transAxes,
#    ha="right", va="top",
#    bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
#)

success_rate = df["project_succeeded"].mean() * 100
#ax.text(
#    0.95, 0.80, f"Project Success Rate: {success_rate:.1f} %",
#    transform=ax.transAxes,
#    ha="right", va="top",
#    bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
#)

#ax.legend(frameon=False)  # legend without a border
plt.tight_layout()
plt.savefig("cumulative_revenue_distribution_gradient_clean.png", dpi=300, bbox_inches='tight', transparent=True, format='png')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'output/simulation_results_plus10price1000_df.csv'